### Iterators, generator expressions and generators

#### Iterators

In [3]:
nums = [1, 2, 3]      # note that ... varies: these are different objects
iter(nums) 
nums.__iter__()  
nums.__reversed__()                  

it = iter(nums)

In [4]:
next(it)

1

In [5]:
next(it)

2

In [8]:
f = open('./1.02 Python basic.ipynb')
f is f.__iter__()
f.close()

#### Generator expressions

In [9]:
(i for i in nums)

<generator object <genexpr> at 0x0000026C9E534620>

In [10]:
[i for i in nums]

[1, 2, 3]

In [11]:
list(i for i in nums)

[1, 2, 3]

In [12]:
{i for i in range(3)}  

{0, 1, 2}

In [13]:
{i:i**2 for i in range(3)}  

{0: 0, 1: 1, 2: 4}

#### Generator

In [16]:
def f():
    yield 1
    yield 2

gen = f()

In [17]:
next(gen)

1

In [18]:
next(gen)

2

In [19]:
next(gen)

StopIteration: 

In [20]:
def f():
    print('start')
    yield 3
    print('middle')
    yield 4
    print('finished')
    yield 5
    
gen = f()
next(gen)

start


3

In [21]:
next(gen)

middle


4

In [22]:
next(gen)

finished


5

In [23]:
next(gen)

StopIteration: 

#### Bidirectional communication

In [41]:
import itertools

def g():
    print('start')
    for i in itertools.count():
        print('yielding %i' % i)
        try:
            ans = yield i
        except GeneratorExit:
            print('closing')
            raise
        except Exception as e:
            print('yield raised %r' % e)
        else:
            print('yield returned %s' % ans)
            
            

In [42]:
it = g()
next(it)

start
yielding 0


0

In [43]:
it.send(11)

yield returned 11
yielding 1


1

In [44]:
it.throw(IndexError)

yield raised IndexError()
yielding 2


2

In [45]:
it.close()

closing


#### Chaining communication

In [48]:
def subgen():
    yield from g()

### Decorators

#### Replacing or tweaking the original object

In [51]:
def simple_decorator(function):
  print("doing decoration")
  return function

@simple_decorator
def function():
  print("inside function")

function()

doing decoration
inside function


In [52]:

def decorator_with_arguments(arg):
  print("defining the decorator")
  def _decorator(function):
      # in this inner function, arg is available too
      print("doing decoration, %r" % arg)
      return function
  return _decorator

@decorator_with_arguments("abc")
def function():
  print("inside function")


function()

defining the decorator
doing decoration, 'abc'
inside function


In [53]:
def replacing_decorator_with_args(arg):
  print("defining the decorator")
  def _decorator(function):
      # in this inner function, arg is available too
      print("doing decoration, %r" % arg)
      def _wrapper(*args, **kwargs):
          print("inside wrapper, %r %r" % (args, kwargs))
          return function(*args, **kwargs)
      return _wrapper
  return _decorator

@replacing_decorator_with_args("abc")
def function(*args, **kwargs):
    print("inside function, %r %r" % (args, kwargs))
    return 14


function(11, 12)

defining the decorator
doing decoration, 'abc'
inside wrapper, (11, 12) {}
inside function, (11, 12) {}


14

#### Decorators implemented as classes and as functions

In [54]:
class decorator_class(object):
  def __init__(self, arg):
      # this method is called in the decorator expression
      print("in decorator init, %s" % arg)
      self.arg = arg
  def __call__(self, function):
      # this method is called to do the job
      print("in decorator call, %s" % self.arg)
      return function
deco_instance = decorator_class('foo')

@deco_instance
def function(*args, **kwargs):
  print("in function, %s %s" % (args, kwargs))

function()

in decorator init, foo
in decorator call, foo
in function, () {}


In [55]:
class replacing_decorator_class(object):
  def __init__(self, arg):
      # this method is called in the decorator expression
      print("in decorator init, %s" % arg)
      self.arg = arg
  def __call__(self, function):
      # this method is called to do the job
      print("in decorator call, %s" % self.arg)
      self.function = function
      return self._wrapper
  def _wrapper(self, *args, **kwargs):
      print("in the wrapper, %s %s" % (args, kwargs))
      return self.function(*args, **kwargs)
deco_instance = replacing_decorator_class('foo')

@deco_instance
def function(*args, **kwargs):
  print("in function, %s %s" % (args, kwargs))

function(11, 12)

in decorator init, foo
in decorator call, foo
in the wrapper, (11, 12) {}
in function, (11, 12) {}


### Copying the docstring and other attributes of the original function

In [56]:
import functools
def replacing_decorator_with_args(arg):
  print("defining the decorator")
  def _decorator(function):
      print("doing decoration, %r" % arg)
      def _wrapper(*args, **kwargs):
          print("inside wrapper, %r %r" % (args, kwargs))
          return function(*args, **kwargs)
      return functools.update_wrapper(_wrapper, function)
  return _decorator
@replacing_decorator_with_args("abc")
def function():
    "extensive documentation"
    print("inside function")
    return 14


function                           

print(function.__doc__)

defining the decorator
doing decoration, 'abc'
extensive documentation


#### Example in the standard library

In [57]:
class Array(object):
    def __init__(self, data):
        self.data = data

    @classmethod
    def fromfile(cls, file):
        data = numpy.load(file)
        return cls(data)

In [60]:
class A(object):
  @property
  def a(self):
    "an important attribute"
    return "a value"
A.a                                   

A().a


'a value'

In [61]:
class Rectangle(object):
    def __init__(self, edge):
        self.edge = edge

    @property
    def area(self):
        """Computed area.

        Setting this updates the edge length to the proper value.
        """
        return self.edge**2

    @area.setter
    def area(self, area):
        self.edge = area ** 0.5

In [62]:
class D(object):
   @property
   def a(self):
     print("getting 1")
     return 1
   @a.setter
   def a(self, value):
     print("setting %r" % value)
   @a.deleter
   def a(self):
     print("deleting")

In [63]:
D.a

In [64]:
D.a.fget

<function __main__.D.a>

In [65]:
D.a.fset

<function __main__.D.a>

In [66]:
D.a.fdel

<function __main__.D.a>

In [67]:
d = D()

In [68]:
d.a

getting 1


1

In [69]:
d.a = 2

setting 2


In [72]:
d.a

getting 1


1

In [70]:
del d.a

deleting


In [71]:
d.a

getting 1


1

#### Deprection of function

In [74]:
class deprecated(object):
    """Print a deprecation warning once on first use of the function.

    >>> @deprecated()                    # doctest: +SKIP
    ... def f():
    ...     pass
    >>> f()                              # doctest: +SKIP
    f is deprecated
    """
    def __call__(self, func):
        self.func = func
        self.count = 0
        return self._wrapper
    def _wrapper(self, *args, **kwargs):
        self.count += 1
        if self.count == 1:
            print(self.func.__name__, 'is deprecated')
        return self.func(*args, **kwargs)

In [75]:
def deprecated(func):
    """Print a deprecation warning once on first use of the function.

    >>> @deprecated                      # doctest: +SKIP
    ... def f():
    ...     pass
    >>> f()                              # doctest: +SKIP
    f is deprecated
    """
    count = [0]
    def wrapper(*args, **kwargs):
        count[0] += 1
        if count[0] == 1:
            print (func.__name__, 'is deprecated')
        return func(*args, **kwargs)
    return wrapper

In [88]:
@deprecated
def func():
    print(1)
    
func()

def func2():
    print(2)
    pass

func3 = deprecated(func2)
func3()


func is deprecated
1
func2 is deprecated
2


In [98]:
class decorator_class(object):
    def __init__(self, arg):
        print('init dec')
        self.arg = arg
    def __call__(self, function):
        print('dec')
        self.function = function
        return self._wrapper
    def _wrapper(self, *args, **kwargs):
        print('wrapper')
        return self.function(*args, **kwargs)
deco_instance = decorator_class('foo')


init dec


In [99]:

@deco_instance
def function(*args, **kwargs):
    print('function')


dec


In [100]:

function()

wrapper
function


#### A while-loop removing decorator

In [101]:
def find_answers():
    answers = []
    while True:
        ans = look_for_next_answer()
        if ans is None:
            break
        answers.append(ans)
    return answers

In [102]:
def vectorized(generator_func):
    def wrapper(*args, **kwargs):
        return list(generator_func(*args, **kwargs))
    return functools.update_wrapper(wrapper, generator_func)

In [103]:
@vectorized
def find_answers():
    while True:
        ans = look_for_next_answer()
        if ans is None:
            break
        yield ans

#### A plugin registration system

In [104]:
class WordProcessor(object):
    PLUGINS = []
    def process(self, text):
        for plugin in self.PLUGINS:
            text = plugin().cleanup(text)
        return text

    @classmethod
    def plugin(cls, plugin):
        cls.PLUGINS.append(plugin)

@WordProcessor.plugin
class CleanMdashesExtension(object):
    def cleanup(self, text):
        return text.replace('&mdash;', u'\N{em dash}')

### Context managers

In [109]:
class closing(object):
    def __init__(self, obj):
        self.obj = obj
    def __enter__(self):
        return self.obj
    def __exit__(self, *args):
        self.obj.close()
    
with closing(open('tmp', 'w')) as f:
    f.write('the contents\n')   
    
with open('tmp', 'a') as f:
  f.write('more contents\n')  

#### Catching exceptions

In [110]:
class assert_raises(object):
    # based on pytest and unittest.TestCase
    def __init__(self, type):
        self.type = type
    def __enter__(self):
        pass
    def __exit__(self, type, value, traceback):
        if type is None:
            raise AssertionError('exception expected')
        if issubclass(type, self.type):
            return True # swallow the expected exception
        raise AssertionError('wrong exception type')

with assert_raises(KeyError):
    {}['foo']

#### Using generators to define context managers

In [112]:
import contextlib
@contextlib.contextmanager
def closing(obj):
    try:
        yield obj
    finally:
        obj.close()

In [113]:
@contextlib.contextmanager
def assert_raises(type):
    try:
        yield
    except type:
        return
    except Exception as value:
        raise AssertionError('wrong exception type')
    else:
        raise AssertionError('exception expected')